## YΣ19 Artificial Intelligence II
# Homework 2

### Iglezou Myrto - 111520170038

# Model 1

In [389]:
import pandas as pd 
from pandas import DataFrame, read_csv

In [390]:
# import gdown
# url = 'https://drive.google.com/file/d/1dTIWNpjlrnTQBIQtaGOh0jCRYZiAQO79/view?usp=sharing'
# path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
# output = "twitterData.csv"
# gdown.download(path, output, quiet=False)
# twitterData = pd.read_csv("/content/twitterData.csv")
# df = pd.DataFrame(data=twitterData)
# df.head(5)

path = r"C:\Users\myrto\Documents\GitHub\AI2-project2\SentimentTweets.csv"
file = pd.read_csv(path,index_col=False,nrows=50000)
# file = pd.read_csv(path,index_col=False)
df = pd.DataFrame(data=file)
df.head(5)

,Unnamed: 0,target,id,date,flag,user,text
0,680949,0,2249621587,Fri Jun 19 22:41:08 PDT 2009,NO_QUERY,sukumarpant,#brokenpromises...
1,406741,0,2059003515,Sat Jun 06 16:03:21 PDT 2009,NO_QUERY,MTMSparrow,David Carradine so sad. Thai's law not sure i...
2,1337108,4,2017466467,Wed Jun 03 08:26:14 PDT 2009,NO_QUERY,itsmemcee,A @ 415 B @ 425. Tell your bro i say congrats!
3,1560887,4,2186457254,Mon Jun 15 18:52:04 PDT 2009,NO_QUERY,jdfreivald,@littlefluffycat Indeed.
4,1466295,4,2064458395,Sun Jun 07 06:19:20 PDT 2009,NO_QUERY,CrazyHan,Completed Race 4 Life in 58mins with girlies f...


In [391]:
df['target'].replace({4: 1}, inplace = True)
X = df['text']
y = df['target']

## Data pre-processing

### Removing the scpecial characters

In [392]:
import re

def RemoveSpecialCharacters(x):
    x = re.sub(r'[^\w]+', " ", x)
    return x

In [393]:
X = X.apply(lambda x: RemoveSpecialCharacters(x) )

### Lemmatize

In [394]:
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

X = X.apply(lambda x: WordNetLemmatizer().lemmatize(x))

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\myrto\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Removing the upercase

In [395]:
X = X.apply(lambda x: x.lower())

### Create a list o all words

In [396]:
words = []
for row in X:
    sentence = row.split()
    for word in sentence:
        words.append(word)
        
words.append("<pad>")

### Create the vocabulary

In [397]:
from collections import Counter

counts = Counter(words)
vocab = sorted(counts, key=counts.get, reverse=True)
word_2_id = {w:i for i,w in enumerate(vocab,1)}

### Encode the words

In [398]:
def convert_tokens_to_inds(sentence, word_2_id):
    return [word_2_id.get(t) for t in sentence]

In [399]:
X = X.apply(lambda x: x.split())

In [400]:
X = X.apply(lambda x: convert_tokens_to_inds(x, word_2_id))

In [401]:
X

0                                                   [194640]
1          [860, 4915, 18, 114, 3405, 12, 1548, 26, 202, ...
2          [4, 32137, 333, 50419, 276, 46, 973, 1, 181, 630]
3                                              [34359, 1064]
4          [3234, 1366, 159, 186, 10, 194641, 23, 5710, 5...
                                 ...                        
1279995    [138603, 141, 5444, 7, 541, 86, 1, 44, 16, 34,...
1279996                                       [9, 424, 3006]
1279997    [186222, 18, 126, 2, 288, 63, 46, 322, 16, 12,...
1279998    [589806, 8, 19, 193, 53, 8, 146, 19389, 9, 4, ...
1279999                [1, 142, 333, 5072, 1, 153, 2684, 12]
Name: text, Length: 1280000, dtype: object

### Check for empty rows

In [402]:
rows = Counter([len(x) for x in X])
print("Zero-length sentences: {}".format(rows[0]))

Zero-length sentences: 0


### Padding for windows.

Find the maximum sentence lenght and make every sentence that long, by adding 0 on the right and left.

In [403]:
def pad_sentence_for_window(sentence, max_row, pad_token=0):
    padding = max_row - len(sentence)
    
    left = (int(padding/2) if padding%2 ==0 else int(padding/2 + 1) )
    right = int(padding/2)
    return [pad_token]*left + sentence + [pad_token]*right

In [404]:
max_row = max(rows)

X = X.apply(lambda x: pad_sentence_for_window(x,max_row))

### Transform data to a 2D array

In [405]:
import numpy as np
X = np.vstack(X.values)

In [406]:
y = y.values.tolist()
y = np.array(y)

### Split data

In [407]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 4, stratify = y)

### Batching sentences together with a DataLoader

In [408]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# Tensor datasets
train_data = TensorDataset(torch.from_numpy(X_train), torch.from_numpy(y_train))
test_data = TensorDataset(torch.from_numpy(X_test), torch.from_numpy(y_test))

# dataloaders
batch_size = 32

# SHUFFLE your data
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=True, batch_size=batch_size)

## Classifier - Model1

In [409]:
import torch.nn as nn

class Model1(nn.Module):
    def __init__(self, vocab_size, input_dim, hidden_dim, output_size, n_layers):
        
        #constructor
        super(Model1,self).__init__()
        
        #embedding layer
#         self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        #dense layer
        self.fc = nn.Linear(input_dim, output_size)
        
        #activation function
        self.sig = nn.Sigmoid()

    def forward(self, x):
#         x = self.embedding(x)
        x = self.fc(x)
        x = self.sig(x)
#         x = self.output(x)
#         x = self.softmax(x)
        return x


### Instantiate the network

In [410]:
vocab_size = len(word_2_id) + 1 # +1 for zero padding + our word tokens
output_size = 1
hidden_dim = 200 
input_dim = X_train.shape[1]
n_layers = 2

net = Model1(vocab_size, input_dim, hidden_dim, output_size, n_layers)

print(net)

Model1(
  (fc): Linear(in_features=119, out_features=1, bias=True)
  (sig): Sigmoid()
)


In [411]:
import torch.optim as optim

lr=0.001
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr)

### Score function 

In [412]:
def compute_accuracy(y_pred, y_target):
    _, y_pred_indices = y_pred.max(dim=1)
    n_correct = torch.eq(y_pred_indices, y_target).sum().item()
    return n_correct / len(y_pred_indices) * 100

## Training

In [413]:
from tqdm import tqdm

num_epochs = 50

list_of_losses = []
list_of_acc = []

for epoch in range(num_epochs):
    running_loss = 0.0
    running_acc = 0.0
    net.train()
    for batch_index, batch in tqdm(enumerate(train_loader)):
        x_train, y_train = batch[0], batch[1]
        x_train = x_train.float()
        y_train = y_train.float()
        
        # step 1. zero the gradients
        optimizer.zero_grad()

        # step 2. compute the output
        y_pred = net(x_train)

        # step 3. compute the loss
        loss = loss_function(y_pred.squeeze(), y_train)
        loss_t = loss.item()
        running_loss += (loss_t - running_loss) / (batch_index + 1)

        # step 4. use loss to produce gradients
        loss.backward()

        # step 5. use optimizer to take gradient step
        optimizer.step()
        # -----------------------------------------
        # compute the accuracy
        acc_t = compute_accuracy(y_pred, y_train)
        running_acc += (acc_t - running_acc) / (batch_index + 1)
   
    list_of_losses.append(running_loss)
    list_of_acc.append(running_acc)


32000it [00:22, 1399.44it/s]
32000it [00:23, 1365.66it/s]
32000it [00:23, 1342.82it/s]
32000it [00:24, 1313.17it/s]
32000it [00:24, 1307.42it/s]
32000it [00:24, 1311.78it/s]
32000it [00:24, 1300.98it/s]
32000it [00:25, 1254.17it/s]
32000it [00:29, 1099.85it/s]
32000it [00:28, 1110.42it/s]
32000it [00:28, 1127.42it/s]
32000it [00:28, 1130.89it/s]
32000it [00:28, 1110.74it/s]
32000it [00:28, 1111.94it/s]
32000it [00:28, 1111.19it/s]
32000it [00:28, 1125.60it/s]
32000it [00:28, 1124.25it/s]
32000it [00:29, 1092.32it/s]
32000it [00:29, 1097.89it/s]
32000it [00:29, 1102.13it/s]
32000it [00:28, 1116.30it/s]
32000it [00:28, 1112.10it/s]
32000it [00:28, 1109.03it/s]
32000it [00:28, 1114.39it/s]
32000it [00:28, 1114.47it/s]
32000it [00:28, 1103.77it/s]
32000it [00:28, 1122.55it/s]
32000it [00:29, 1100.58it/s]
32000it [00:28, 1112.78it/s]
32000it [00:28, 1103.71it/s]
32000it [00:28, 1111.53it/s]
32000it [00:28, 1106.58it/s]
32000it [00:28, 1108.16it/s]
32000it [00:28, 1107.52it/s]
32000it [00:28

In [414]:
print(list_of_acc)

[49.952441406249655, 49.95244140625021, 49.952441406249726, 49.95244140624972, 49.95244140624968, 49.95244140624991, 49.95244140624968, 49.95244140624981, 49.952441406250195, 49.952441406249655, 49.95244140625012, 49.95244140624992, 49.95244140624985, 49.95244140625006, 49.95244140624995, 49.95244140624992, 49.952441406250074, 49.95244140625004, 49.952441406250024, 49.95244140625006, 49.9524414062498, 49.95244140625003, 49.952441406250095, 49.95244140624995, 49.95244140625011, 49.95244140624995, 49.95244140624979, 49.952441406250365, 49.9524414062503, 49.95244140624983, 49.952441406249875, 49.95244140625035, 49.952441406250244, 49.95244140625009, 49.95244140624988, 49.952441406250045, 49.95244140625014, 49.952441406250045, 49.95244140624972, 49.95244140624983, 49.952441406249974, 49.952441406250145, 49.952441406249854, 49.952441406250294, 49.952441406250074, 49.952441406249505, 49.952441406250095, 49.9524414062499, 49.952441406249854, 49.95244140625009]


In [415]:
print(list_of_losses)

[0.48858447833871343, 0.49075259127654053, 0.4928168434780087, 0.49640740413498297, 0.5004110684059596, 0.5004999989084924, 0.50049999860674, 0.5004999981587775, 0.5004999959562074, 0.5004999943152052, 0.5004013557033564, 0.49844354333635443, 0.49700209361361214, 0.5001009447239321, 0.4995107389180896, 0.49956578909000626, 0.4990217291382119, 0.49958791788062307, 0.500020136715376, 0.49841186832310536, 0.4987901349938466, 0.4998729211436594, 0.4994963711537407, 0.4992566777402534, 0.4997555513870884, 0.49929302887758287, 0.4982258266801959, 0.499344069693236, 0.4992588027045134, 0.49822243971983127, 0.49846446063089983, 0.4985831323545414, 0.4991029941919239, 0.4981015524761762, 0.4989248820347739, 0.5004273619567977, 0.4981387082841249, 0.49955207099672344, 0.4986564508969873, 0.49968907251022743, 0.4994264059653498, 0.4996543103614839, 0.49814619185868597, 0.4983540568193412, 0.49819209470506975, 0.49814723806129596, 0.49797411802876745, 0.49791086177854027, 0.49784772854158577, 0.49

## Testing

In [416]:
from sklearn.metrics import accuracy_score
accuracy = []
for bidx, batch in tqdm(enumerate(test_loader)):
    x_test, y_test = batch[0], batch[1]
    x_test = x_test.float()
    y_test = y_test.float()
    output = net.forward(x_test)
    acc = compute_accuracy(output, y_test)
    accuracy.append(acc)

8000it [00:03, 2089.45it/s]


In [417]:
def Average(lst): 
    return sum(lst) / len(lst) 
  
average = Average(accuracy) 

In [418]:
average

49.95234375